# FCC Application Scraping

## Set up selenium webdriver with Firefox
* Download Firefox driver [geckodriver](https://github.com/mozilla/geckodriver/releases) for selenium
* Move the driver executable into the same directory as this notebook (or update the path to the executable in `scrape_table_for_call_sign`)
* `pip install selenium`

-- see scrape-apps_by_callsign.ipynb for more details

In [1]:
from bs4 import BeautifulSoup
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options  
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException        

## Scraping FCC tables for an individual call sign

def element_exists(driver, xpath):
    """
    Helper function that checks if the element at the given xpath exists on
    driver's current page. This is used to check if there is a "Next" button
    -- to scrape through multiple pages of the FCC's paginated tables.
    """
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


def scrape_table_page(innerHTML):
    """
    Extracts the following details from a call sign table page:
        * File Number
        * Call Sign/Lease ID
        * Applicant Name
        * FRN
        * Purpose - split into Purpose_Main and Purpose_Note
        * Radio
        * Service
        * Receipt
        * Date
        * Status
    Returns as a list of lists.
    """
    soup = BeautifulSoup(innerHTML, "lxml")
    table_page = soup.find("table", {"summary": "Application search results"})
    
    # run through each row
    rows = table_page.find_all("tr")[1:-1]
    table_data = []
    for row in rows:  # note: skip the first (header) and last (pagination) rows
        
        # run through each cell of each row
        cols = row.find_all("td")
        row_data = []
        for i in range(1, len(cols)): # note: skip the first (index) cell
            
            # extract cell data, trim leading/trailing whitespace
            cell_data = cols[i].get_text().strip()
            
            # fifth-index cells "Purpose" has to get split into two cols
            if i == 5:
                
                purpose = cell_data.split("\n")
                purpose_main = purpose[0].strip()
                purpose_note = ""
                
                # sometimes the "Purpose" column has an optional note
                if len(purpose) >= 2:
                    purpose_note = purpose[-1].strip()
                    
                row_data.append(purpose_main)
                row_data.append(purpose_note)
            
            # other cells can be inserted as normal
            else:
                row_data.append(cell_data)
                
        table_data.append(row_data)
        
    return(table_data)


def scrape_table_for_cma(cma, without_browser=False):
    """
    Scrapes the (Market Based) License Search database for a given Cellular Market Area (cma).
    
    Returns the table as a list of lists, for example:
    [
      ['File Number', 'Call Sign/Lease ID', 'Applicant Name', 'FRN', 'Purpose_Main', 'Purpose_Note', 'Radio Service', 'Receipt Date', 'Status'],
      ['0000214527',  'KNKN555',            'AT&T',           '',    'Amendment',    'Transfer Control', 'AL',        '09/05/2000',   'Granted'],
      ...,
      ...
    ]
    
    """
    tick = time.time()
    print("LOG[INFO]: START %s " % call_sign)
    
    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    # (optional) don't open up browser
    firefox_options = Options()
    if without_browser:
        firefox_options.add_argument("--headless")
    
    # spin up the webdriver
    firefox = webdriver.Firefox(options=firefox_options, executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/UlsSearch/searchMarket.jsp")
    
    # specify the market type we want to search for
    in_type = Select(firefox.find_element_by_id("marketType"))
    in_type.select_by_value("CMA")
    
    # specify the market we want to search for
    in_mkt = Select(firefox.find_element_by_id("ulsMarket"))
    in_mkt.select_by_value("CMA001")

    # set pagination to 100 (maximum)
    pagination = Select(firefox.find_element_by_xpath("//select[@name='pageSize']"))
    pagination.select_by_visible_text("100")
    
    # submit the search form, wait for load
    firefox.find_element_by_name("search").submit()
    WebDriverWait(firefox, 15).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
    WebDriverWait(firefox, 15).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
    
    table_data = []
    
    # scrape the table on the first page
    innerHTML = firefox.execute_script("return document.body.innerHTML")
    table_data += scrape_table_page(innerHTML)
    
    # as long there is a link to "next page"...
    next_page_xpath = "//*[@title='Next page of results']"
    while element_exists(firefox, next_page_xpath):
        # ... go to next page
        firefox.find_element_by_xpath(next_page_xpath).click()
        # ... wait for load
        WebDriverWait(firefox, 15).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
        # .. and scrape
        innerHTML = firefox.execute_script("return document.body.innerHTML")
        table_data += scrape_table_page(innerHTML)

    # gracefully close the webdriver
    firefox.quit()
    
    tock = time.time()
    print("LOG[INFO]: END %s (%2f seconds)" % (call_sign, tock - tick))
    
    return table_data
     

In [3]:
def scrape_table_for_cma(mkt_num, without_browser=False):

    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    # (optional) don't open up browser
    firefox_options = Options()
    if without_browser:
        firefox_options.add_argument("--headless")

    # spin up the webdriver
    firefox = webdriver.Firefox(options=firefox_options, executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/UlsSearch/searchMarket.jsp")

    # specify the market type we want to search for
    in_markettype = Select(firefox.find_element_by_id("marketType"))
    in_markettype.select_by_value("CMA")

    # specify the market we want to search for
    in_mkt = Select(firefox.find_element_by_id("ulsMarket"))
    in_mkt.deselect_by_visible_text("All")
    in_mkt.select_by_value(mkt_num)

    # submit search, wait for load
    firefox.find_element_by_name("marketSearch").submit()
    WebDriverWait(firefox, 15).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
    WebDriverWait(firefox, 15).until(lambda _: firefox.execute_script("return document.readyState") == "complete")

In [ ]:
def get_mktsearch_results(mkt_num)
    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    # (optional) don't open up browser
    firefox_options = Options()
    if without_browser:
        firefox_options.add_argument("--headless")

    # spin up the webdriver
    firefox = webdriver.Firefox(options=firefox_options, executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/UlsSearch/searchMarket.jsp")

    # specify the market type we want to search for
    in_markettype = Select(firefox.find_element_by_id("marketType"))
    in_markettype.select_by_value("CMA")

    # specify the market we want to search for
    in_mkt = Select(firefox.find_element_by_id("ulsMarket"))
    in_mkt.deselect_by_visible_text("All")
    in_mkt.select_by_value(mkt_num)

    # submit search, wait for load
    firefox.find_element_by_name("marketSearch").submit()

In [4]:
scrape_table_for_cma("CMA001")
firefox.quit()

NameError: name 'firefox' is not defined

## Scrape the FCC tables for a list of call signs

In [12]:
# read in the list of FCC callsigns
call_signs = ["KNKN555", "KNKN556"]

In [13]:
# collects the scraped data for *all* the call signs into a single python list
scraped_data = []
for cs in call_signs:
    scraped_data.append(scrape_table_for_call_sign(cs, without_browser=False))

NameError: name 'scrape_table_for_call_sign' is not defined

In [18]:
# sanity check over data

In [54]:
# print first 10 rows
for row in scraped_data[:10]:
    print(row)

['0000083337', '', 'AT&T Wireless Services of Florida, Inc.', '', 'Assignment of Authorization', '', 'AL', '02/18/2000', 'Consummated']
['0000214527', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Renewal/Modification', '', 'CL', '09/05/2000', 'Granted']
['0000287045', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Duplicate License', '', 'CL', '12/08/2000', 'Granted']
['0000326671', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Duplicate License', '', 'CL', '01/16/2001', 'Granted']
['0000446751', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Administrative Update', '', 'CL', '05/02/2001', 'Granted']
['0000545823', '', 'AT&T Wireless Services, Inc.', '0004122032', 'Transfer of Control', '', 'AL', '08/01/2001', 'Inactive']
['0000545823', '', 'AT&T Wireless Services, Inc.', '0004122032', 'Amendment', '(Transfer of Control)', 'AL', '12/20/2001', 'Granted']
['0000949139', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Modificat